In [27]:
import sys

print(sys.version)

3.9.13 (main, Aug 25 2022, 23:26:10) 
[GCC 11.2.0]


In [28]:
import pandas as pd
from random import randint

import numpy as np
# from scipy import spatial
import random

In [29]:
data = pd.read_csv('./ratings.dat', sep='::', header = None)  # tab-separated data

/tmp/ipykernel_198423/2193334887.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('./ratings.dat', sep='::', header = None)  # tab-separated data


In [30]:
data.columns=['User_ID','movie_ID','rating','timestamp']

In [31]:
data.drop('timestamp', axis=1, inplace=True)

In [32]:
data = data.head(2200)

In [33]:
data

,User_ID,movie_ID,rating
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0
3,1,292,5.0
4,1,316,5.0
...,...,...,...
2195,19,50,5.0
2196,19,62,4.0
2197,19,74,5.0
2198,19,85,2.0


In [34]:
# Get unique User_IDs and movie_IDs
unique_users = data['User_ID'].unique()
unique_items = data['movie_ID'].unique()

In [35]:
# Number of users and items
n_users = len(unique_users)
n_items = len(unique_items)
print(n_users)
print(n_items)

18
1406


In [36]:

# Initialize R matrix with zeros
R = pd.DataFrame(0, index=unique_users, columns=unique_items, dtype=float)
RR = pd.DataFrame(0, index=unique_users, columns=unique_items, dtype=float)

# Fill R matrix based on the dataset
for index, row in data.iterrows():
    user_id = row['User_ID']
    item_id = row['movie_ID']
    # If there's a rating provided, set the corresponding entry to 1
    # if not pd.isnull(row['rating']):
    R.loc[user_id, item_id] = 1
    RR.loc[user_id, item_id] = row['rating']

In [37]:
R.reset_index(drop=True, inplace=True)
RR.reset_index(drop=True, inplace=True)

In [38]:
R.columns = range(len(R.columns))
RR.columns = range(len(RR.columns))

In [39]:
R = R.values
RR = RR.values

In [40]:
unique_users

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19])

In [41]:
# Define the shape of the matrix
rows = R.shape[1]
cols = R.shape[1]

# Generate the zero matrix with values between 0 and 1
S = np.random.rand(rows, cols)


In [42]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 3
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

3

In [43]:
def constraint(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i==j:
                matrix[i,j] = 0
    return matrix

In [44]:
S = constraint(S)
for pu in range(num_clusters):
    S_pu_list[pu] = constraint(S_pu_list[pu])

In [45]:
# Print the list of random matrices
for i, S_pu in enumerate(S_pu_list):
    print(f"S_pu matrix for cluster {i+1}:\n{S_pu}\n")

S_pu matrix for cluster 1:
[[0.         0.82867463 0.34519503 ... 0.67568939 0.10641871 0.78159778]
 [0.93929926 0.         0.22329622 ... 0.26108711 0.57038464 0.67985581]
 [0.93210803 0.16342133 0.         ... 0.88368869 0.28313736 0.24943378]
 ...
 [0.1463321  0.30088542 0.48306465 ... 0.         0.8502947  0.14283493]
 [0.38528735 0.98698222 0.80245514 ... 0.92032022 0.         0.41439525]
 [0.415272   0.08312712 0.5926419  ... 0.13250954 0.35433125 0.        ]]

S_pu matrix for cluster 2:
[[0.         0.88604678 0.3202995  ... 0.73899099 0.88565231 0.30704747]
 [0.10098696 0.         0.69826795 ... 0.29233706 0.81022877 0.32161982]
 [0.50446307 0.02874567 0.         ... 0.50164469 0.66123893 0.69408594]
 ...
 [0.81461416 0.92935243 0.22396142 ... 0.         0.4817775  0.40160614]
 [0.05768177 0.01092868 0.98251071 ... 0.26937088 0.         0.41553374]
 [0.58935516 0.35484307 0.74087855 ... 0.01435967 0.16454854 0.        ]]

S_pu matrix for cluster 3:
[[0.         0.72048722 0.939

In [46]:
def initial_clustering(n_clusters, users):
  random.shuffle(users)  # Shuffle the list randomly
  sublists = []
  size_sublist = len(users) //n_clusters  # Determine the size of each sublist

  # Split the shuffled list into n sublists
  for i in range(n_clusters):
      start = i * size_sublist
      end = (i + 1) * size_sublist if i < n_clusters - 1 else None
      sublists.append(users[start:end])
  return sublists

In [47]:
def compute_gu(pu,u):
  numerator = 0
  denomenator = 0
  for item in range(n_items):

    multiplier1 = 0
    for ind, Ruitem in enumerate(R[u,:]):
      if Ruitem == 1:
        multiplier1 += S[ind, item] - S_pu_list[pu][ind, item]
        denomenator += (S[ind, item] - S_pu_list[pu][ind, item])**2
    multiplier2 = R[u, item]
    for ind, Ruitem in enumerate(R[u,:]):
      if Ruitem == 1:
        multiplier2 -= S_pu_list[pu][ind, item]

    numerator += multiplier1*multiplier2
  return numerator/denomenator


In [48]:
def soft_thresholding(vect, lamb):
    for index, item in enumerate(vect):
        vect[index] = np.maximum(np.sign(item)*(np.abs(item)- lamb),0)
    return vect

In [49]:
no_of_cluster=3

In [50]:
def estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(Rpu_list[i])

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = Rpu_list[pu][:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

In [51]:
def lslim(S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 3
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, S_pu_list

In [52]:
clusters, S_pu_list = lslim(S_pu_list)

initial cluster:  [[3, 12, 14, 1, 6, 10], [11, 4, 17, 15, 9, 16], [7, 5, 13, 0, 8, 2]]
0.0


In [53]:
clusters

[[3, 12, 14, 1, 6, 10], [11, 4, 17, 15, 9, 16], [7, 5, 13, 0, 8, 2]]

In [54]:
n_clusters=3
Rpu_list = []
for cluster in clusters:
    Rpu = np.zeros((n_users, n_items))
    for user in cluster:
        for ind, item in enumerate(R[user, :]):
            if item == 1:
                Rpu[user, ind] = 1
    Rpu_list.append(Rpu)

In [55]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

[7, 31, 68, 84, 79, 34, 223, 678, 998, 1009, 40, 1096, 760, 1230, 525, 1267, 1324, 73]


In [56]:
for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [57]:
#run glslim on this data, get S and Spu matrices
n_clusters = 3
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:

      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [58]:
hits = 0
N = 20
for user in range(n_users):
    recommended_items = [t[0] for t in user_non_rated_item_predicted_ratings[user][:N]]
    if random_indices[user] in recommended_items:
        hits += 1

hit_rate = hits / n_users

print("Hit rate:", hit_rate)

Hit rate: 0.0


In [59]:
# For 4 clusters

In [60]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 4
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

4

In [61]:
def constraint(matrix):
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i==j:
                matrix[i,j] = 0
    return matrix

In [62]:
S = constraint(S)
for pu in range(num_clusters):
    S_pu_list[pu] = constraint(S_pu_list[pu])

In [63]:
def estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, betal, lambdal):
    newSpu = []
    for m in range(len(S_pu_list)):
        newSpu.append(np.zeros((S_pu_list[m].shape[0], S_pu_list[m].shape[1])))
    Rpu_list = []
    for cluster in clusters:
        Rpu = np.zeros((n_users, n_items))
        for user in cluster:
            for ind, item in enumerate(R[user, :]):
                if item == 1:
                    Rpu[user, ind] = 1
        Rpu_list.append(Rpu)
    g = gu_users.reshape(-1,1)
    gdash = 1-g
    A = np.multiply(g,R)
    B=[]
    for i in range(n_clusters):
        B.append(Rpu_list[i])

    for item in range(n_items):
        for pu in range(n_clusters):
            ri  = Rpu_list[pu][:, item]
            si = S[:, item]
            leftsideofSoftthresholding = ((B[pu].T@ri)).reshape((n_items, 1))
            summationofspui = np.zeros((n_users, 1))
            for j in range(n_clusters):
                if j != pu:
                    summationofspui += ((B[j])@(S_pu_list[j][:, item].reshape((n_items,1)))).reshape((n_users, 1))
            leftsideofSoftthresholding -= B[pu].T@summationofspui
            rightsideofSofthreholding = lambdal
            Softthresholdingoutput = soft_thresholding(leftsideofSoftthresholding, rightsideofSofthreholding)
            newSpu[pu][:,item] = (np.linalg.inv(betal*np.identity(n_items) + B[pu].T@B[pu])@(Softthresholdingoutput).flatten())
    for pu in range(num_clusters):
        newSpu[pu] = constraint(newSpu[pu])
    return newSpu

In [64]:
def lslim(S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 4
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, S_pu_list

In [65]:
clusters, S_pu_list = lslim(S_pu_list)

initial cluster:  [[14, 2, 0, 8], [7, 11, 15, 12], [10, 3, 1, 13], [4, 17, 6, 9, 16, 5]]
0.0


In [66]:
clusters

[[14, 2, 0, 8], [7, 11, 15, 12], [10, 3, 1, 13], [4, 17, 6, 9, 16, 5]]

In [67]:
n_clusters=4
Rpu_list = []
for cluster in clusters:
    Rpu = np.zeros((n_users, n_items))
    for user in cluster:
        for ind, item in enumerate(R[user, :]):
            if item == 1:
                Rpu[user, ind] = 1
    Rpu_list.append(Rpu)

In [68]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

[12, 31, 45, 19, 104, 207, 236, 353, 704, 1017, 48, 605, 1117, 505, 137, 192, 707, 99]


In [69]:
for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [70]:
#run glslim on this data, get S and Spu matrices
n_clusters = 4
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:

      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [71]:
hits = 0
N = 30
for user in range(n_users):
    recommended_items = [t[0] for t in user_non_rated_item_predicted_ratings[user][:N]]
    if random_indices[user] in recommended_items:
        hits += 1

hit_rate = hits / n_users

print("Hit rate:", hit_rate)

Hit rate: 0.0


In [46]:
#For 5 clusters

In [62]:
def generate_S_pu(num_clusters, rows):
    S_pu_list = []
    for _ in range(num_clusters):
        S_pu = np.random.rand(rows, rows)  # Generate zero matrix with values between 0 and 1
        S_pu_list.append(S_pu)
    return S_pu_list

# Define parameters
num_clusters = 5
# Generate random matrices for each cluster
S_pu_list = generate_S_pu(num_clusters, rows)
len(S_pu_list)

5

In [63]:
def lslim(S_pu_list):
  gu_users = np.array([0.5 for u in range(n_users)])
  n_clusters = 5
  clusters = initial_clustering(n_clusters, list(range(n_users)))
  print("initial cluster: ", clusters)
  while True:
    count = 0
    S_pu_list =estimate_Spu(n_clusters, clusters, gu_users,S_pu_list, 0.2, 0.1)
    for u in list(range(n_users)):
      storetrainingerror = []

      for pu in range(n_clusters):
        gu = compute_gu(pu, u)

        modR = sum(R[u,:])
        training_error = 0
        for i, Ruitem in enumerate(R[u,:]):
          if Ruitem == 1:
            diff = RR[u, i]
            rui = 0
            for l, Ruitem in enumerate(R[u,:]):
              if Ruitem == 1:
                rui += S_pu_list[pu][l,i]
            diff -= rui
            training_error+= diff**2
        training_error /= modR
        training_error = np.sqrt(training_error)
        storetrainingerror.append(training_error)
      argminpu = np.argmin(storetrainingerror)
      if u not in clusters[argminpu]:
        for k in range(n_clusters):
          if u in clusters[k]:
            clusters[k].remove(u)
        clusters[argminpu].append(u)
        count +=1
      gu_users[u] = gu
    print(count/n_users)
    if count/n_users <= 0.05:
      break
  return clusters, S_pu_list

In [64]:
clusters, S_pu_list = lslim(S_pu_list)

initial cluster:  [[6, 5, 4], [16, 1, 8], [3, 2, 10], [0, 13, 7], [15, 11, 14, 9, 12]]
0.0


In [65]:
n_clusters=5
Rpu_list = []
for cluster in clusters:
    Rpu = np.zeros((n_users, n_items))
    for user in cluster:
        for ind, item in enumerate(R[user, :]):
            if item == 1:
                Rpu[user, ind] = 1
    Rpu_list.append(Rpu)

In [66]:
# Sample NumPy matrix
matrix = np.copy(R)

# Initialize list to store random indices
random_indices = []

# Iterate over each row
for row in matrix:
    # Get indices of non-zero elements in the row
    nonzero_indices = np.nonzero(row)[0]
    if len(nonzero_indices) > 0:
        # Choose one random index from the non-zero indices
        random_index = np.random.choice(nonzero_indices)
        random_indices.append(random_index)

print(random_indices)

[3, 25, 59, 78, 97, 190, 289, 698, 992, 1032, 1084, 1096, 141, 1187, 14, 1190, 1323]


In [67]:
for ind, row in enumerate(matrix):
  row[random_indices[ind]] = 0

In [68]:
#run glslim on this data, get S and Spu matrices
n_clusters = 5
user_non_rated_item_predicted_ratings = []
for u in list(range(n_users)):
  user_store_for_each_item = []
  for i, Ruitem in enumerate(matrix[u,:]):
    rui = 0
    if Ruitem == 0.0:

      user_pu = 0
      for pu in range(n_clusters):
          if u in clusters[pu]:
            user_pu = pu
      for l, Ruitem in enumerate(R[u,:]):
        if Ruitem == 1:
          rui += S_pu_list[user_pu][l,i]
    user_store_for_each_item.append((i, rui))
  user_store_for_each_item = sorted(user_store_for_each_item, key=lambda x: x[0], reverse=True)
  user_non_rated_item_predicted_ratings.append(user_store_for_each_item)

In [69]:
hits = 0
N = 20
for user in range(n_users):
    recommended_items = [t[0] for t in user_non_rated_item_predicted_ratings[user][:N]]
    if random_indices[user] in recommended_items:
        hits += 1

hit_rate = hits / n_users

print("Hit rate:", hit_rate)

Hit rate: 0.058823529411764705
